In [ ]:
# | default_exp utils

# Imports

In [ ]:
# | export
import json
import os
import shutil
import subprocess
import sys
import threading
from collections import Counter, defaultdict
from pathlib import Path

import matplotlib.pyplot as plt
from matplotlib import cm
from torch.utils import data

plt.style.use("seaborn-v0_8-whitegrid")  # ou "ggplot", "seaborn-paper", etc.

plt.rcParams.update(
    {
        "font.size": 16,
        "axes.titlesize": 20,
        "axes.labelsize": 18,
        "legend.fontsize": 14,
        "xtick.labelsize": 14,
        "ytick.labelsize": 14,
        "font.family": "serif",  # ou "Times New Roman"
    }
)
import numpy as np
import open3d as o3d
import yaml

# Dataset

In [ ]:
# | export
class Dataset(data.Dataset):
    def __init__(
        self,
        data_path: str,
        split: str = "train",
        load_cluster: bool = False,
        return_np_array: bool = True,
    ) -> None:
        """
        Initialize dataset loader.

        Args:
            data_path (str or Path): Base path to the SemanticKITTI dataset.
            split (str): Dataset split to use ('train', 'valid', or 'test').
        """
        self.data_path: Path = Path(data_path)
        self.split: str = split
        self.is_test: bool = split == "test"
        self.load_cluster: bool = load_cluster
        self.return_np_array = return_np_array

        # Paths to YAML config and data folders
        self.yaml_path: Path = Path("../tests/semantic-kitti.yaml")
        # self.yaml_path: Path = self.data_path / 'semantic-kitti.yaml'
        self.velodynes_path: Path = (
            self.data_path / "data_odometry_velodyne/dataset/sequences"
        )
        self.labels_path: Path = (
            self.data_path / "data_odometry_labels/dataset/sequences"
        )
        self.clusters_path: Path = (
            self.data_path / "data_odometry_clusters/dataset/sequences"
        )

        # Load dataset metadata and label mappings
        with open(self.yaml_path, "r") as file:
            metadata: dict = yaml.safe_load(file)

        self.sequences: list[int] = metadata["split"][split]
        self.learning_map: dict[int, int] = metadata["learning_map"]

        # Convert label map to numpy for fast lookup
        max_label: int = max(self.learning_map.keys())
        self.learning_map_np: np.ndarray = np.zeros((max_label + 1,), dtype=np.uint32)
        for raw_label, mapped_label in self.learning_map.items():
            self.learning_map_np[raw_label] = mapped_label

        # Collect all frame paths for selected sequences
        self.frame_paths: list[tuple[str, str]] = self._collect_frame_paths()

        self.last_seq = None
        self.last_frame_id = None

    def _collect_frame_paths(self) -> "list[tuple[str, str]]":
        """Collect all (sequence, frame_id) pairs from the dataset split."""
        frame_list = []
        for seq in self.sequences:
            seq_str = f"{int(seq):02d}"
            seq_velo_path = self.velodynes_path / seq_str / "velodyne"
            velo_files = sorted(seq_velo_path.glob("*.bin"))
            for file in velo_files:
                frame_list.append((seq_str, file.stem))
        return frame_list

    def __len__(self) -> int:
        """Return number of samples in the dataset split."""
        return len(self.frame_paths)

    def __iter__(self):
        self._iter_idx = 0
        return self

    def __next__(self):
        if self._iter_idx < len(self):
            item = self[self._iter_idx]
            self._iter_idx += 1
            return item
        else:
            raise StopIteration

    def _compute_scanline_ids(
        self, point_cloud: np.ndarray, n_scans: int = 64
    ) -> np.ndarray:
        """
        Approximate scanline indices based on point order.

        Args:
            point_cloud (np.ndarray): Nx3 array of 3D points.
            n_scans (int): Number of LiDAR scanlines (e.g., 64 for HDL-64E).

        Returns:
            np.ndarray: Nx1 array with estimated scanline indices (0 to n_scans - 1).
        """
        total_points = point_cloud.shape[0]
        scanline_ids = np.floor(
            np.linspace(0, n_scans, total_points, endpoint=False)
        ).astype(int)
        return scanline_ids.reshape(-1, 1)

    def get_last_seq_frame(self) -> tuple[str, str] | None:
        """
        Get the last sequence and frame ID loaded.

        Returns:
            tuple: (sequence, frame_id)
        """

        if self.last_seq is None or self.last_frame_id is None:
            return None
        else:
            return self.last_seq, self.last_frame_id

    def __getitem__(self, idx: int) -> "dict[str, np.ndarray] | np.ndarray":
        """
        Load a sample from the dataset.

        Args:
            idx (int): Index of the frame to load.

        Returns:
            tuple:
                - point_cloud_with_label (np.ndarray): Nx6 array [x, y, z, true_label, pred_label, scanline_id].
                - item_dict (dict): Contains 'point_cloud', 'label', and 'mask'.
        """

        seq, frame_id = self.frame_paths[idx]
        self.last_seq = seq
        self.last_frame_id = frame_id

        # Load point cloud (Nx4), drop reflectance
        velodyne_file_path = self.velodynes_path / seq / "velodyne" / f"{frame_id}.bin"
        with open(velodyne_file_path, "rb") as file:
            point_cloud = np.fromfile(file, dtype=np.float32).reshape(-1, 4)[:, :3]

        # Load and map semantic labels
        if not self.is_test:
            label_file_path = self.labels_path / seq / "labels" / f"{frame_id}.label"
            if label_file_path.exists():
                with open(label_file_path, "rb") as file:
                    raw_labels = np.fromfile(file, dtype=np.uint32) & 0xFFFF
                labels = self.learning_map_np[raw_labels]
                mask = labels != 0
            else:
                labels = np.zeros(point_cloud.shape[0], dtype=np.uint32)
                mask = np.ones(point_cloud.shape[0], dtype=bool)
        else:
            labels = np.zeros(point_cloud.shape[0], dtype=np.uint32)
            mask = np.ones(point_cloud.shape[0], dtype=bool)

        # Estimate scanline indices
        scanline_ids = self._compute_scanline_ids(point_cloud)

        if self.load_cluster:
            cluster_file_path = (
                self.clusters_path / seq / "clusters" / f"{frame_id}_cluster.label"
            )
            if cluster_file_path.exists():
                with open(cluster_file_path, "rb") as file:
                    clusters = np.fromfile(file, dtype=np.uint32)
                if clusters.shape[0] != point_cloud.shape[0]:
                    raise ValueError(
                        f"Predicted label count mismatch for frame {seq}/{frame_id}"
                    )
                clusters = clusters.reshape(-1, 1)
            else:
                clusters = np.zeros((point_cloud.shape[0], 1), dtype=np.uint32)
        else:
            clusters = np.zeros((point_cloud.shape[0], 1), dtype=np.uint32)

        # # Final format: [x, y, z, true_label, predicted_label, scanline_id]
        point_cloud_array = np.hstack(
            (point_cloud, labels.reshape(-1, 1), clusters, scanline_ids)
        )

        item_dict = {
            "point_cloud": point_cloud,
            "label": labels,
            "cluster": clusters,
            "scanline_id": scanline_ids,
            "mask": mask,
        }

        if self.return_np_array:
            return point_cloud_array
        else:
            return item_dict

# Visualizer for Point Clouds

In [ ]:
# | export
class PointCloudVisualizer:
    def __init__(
        self,
        point_size: float = 1.0,
        grid_size=50,
        grid_spacing=1.0,
        grid_line_width=10,
        cluster_viz=False,
    ):
        """
        Visualizer class for rendering point clouds using Open3D with color-coded semantic labels.

        Args:
            point_size (float): Default size of points in the Open3D viewer.
        """
        self.point_size = point_size
        self.fixed_colors_rgb = self._get_fixed_colors_rgb()
        self.grid_size = grid_size
        self.grid_spacing = grid_spacing
        self.grid_line_width = grid_line_width
        self.cluster_viz = cluster_viz

    def set_point_sieze(self, point_size):
        self.point_size = point_size

    def set_grid_size(self, grid_size):
        self.grid_size = grid_size

    def set_grid_spacing(self, grid_spacing):
        self.grid_spacing = grid_spacing

    def set_grid_line_width(self, grid_line_width):
        self.grid_line_width = grid_line_width

    def _get_fixed_colors_rgb(self) -> "dict[int, list[float]]":
        fixed_colors = {
            -1: [255, 255, 255],  # plane
            0: [0, 0, 0],  # unlabeled
            1: [245, 150, 100],  # car
            2: [245, 230, 100],  # bicycle
            3: [150, 60, 30],  # motorcycle
            4: [180, 30, 80],  # truck
            5: [250, 80, 100],  # other-vehicle
            6: [30, 30, 255],  # person
            7: [200, 40, 255],  # bicyclist
            8: [90, 30, 150],  # motorcyclist
            9: [255, 0, 255],  # road
            10: [255, 150, 255],  # parking
            11: [75, 0, 75],  # sidewalk
            12: [75, 0, 175],  # other-ground
            13: [0, 200, 255],  # building
            14: [50, 120, 255],  # fence
            15: [0, 175, 0],  # vegetation
            16: [0, 60, 135],  # trunk
            17: [80, 240, 150],  # terrain
            18: [150, 240, 255],  # pole
            19: [0, 0, 255],  # traffic-sign
        }
        return {
            label: [c / 255.0 for c in reversed(rgb)]
            for label, rgb in fixed_colors.items()
        }

    def _get_color_map(self, labels: np.ndarray) -> np.ndarray:
        """
        Assigns RGB colors to labels.

        Args:
            labels (np.ndarray): Array of label ids.

        Returns:
            np.ndarray: Nx3 array of RGB colors.
        """
        color_map = np.zeros((labels.shape[0], 3))
        for i, label in enumerate(labels):
            if label in self.fixed_colors_rgb:
                color_map[i] = self.fixed_colors_rgb[label]
            else:
                rng = np.random.default_rng(label)
                color_map[i] = rng.random(3)
        return color_map

    def _create_grid(self, grid_size=50, grid_spacing=1.0):
        points = []
        lines = []
        colors = []

        for i in range(-grid_size, grid_size + 1):
            # Linhas paralelas ao eixo X
            points.append([i * grid_spacing, -grid_size * grid_spacing, 0])
            points.append([i * grid_spacing, grid_size * grid_spacing, 0])
            lines.append([len(points) - 2, len(points) - 1])

            # Linhas paralelas ao eixo Y
            points.append([-grid_size * grid_spacing, i * grid_spacing, 0])
            points.append([grid_size * grid_spacing, i * grid_spacing, 0])
            lines.append([len(points) - 2, len(points) - 1])

            # Cores: mais clara a cada 10 unidades
            color = [0.3, 0.3, 0.3] if i % 10 else [0.6, 0.6, 0.6]
            colors.extend([color, color])

        grid = o3d.geometry.LineSet()
        grid.points = o3d.utility.Vector3dVector(points)
        grid.lines = o3d.utility.Vector2iVector(lines)
        grid.colors = o3d.utility.Vector3dVector(colors)
        return grid

    def _create_plane(self, normal_d_tuple, size=100.0):
        """
        Cria um plano baseado no vetor normal e no valor d.

        Args:
            normal_d_tuple (tuple): Tuple contendo o vetor normal (x, y, z) e o valor d.
            size (float): Tamanho do plano a ser desenhado.

        Returns:
            o3d.geometry.TriangleMesh: Mesh do plano.
        """
        normal, d = normal_d_tuple

        # Normalizar o vetor normal
        normal = np.array(normal)
        normal = normal / np.linalg.norm(normal)

        # Calcular 4 pontos do plano
        p1 = np.array(
            [-size, -size, -(normal[0] * (-size) + normal[1] * (-size) + d) / normal[2]]
        )
        p2 = np.array(
            [size, -size, -(normal[0] * size + normal[1] * (-size) + d) / normal[2]]
        )
        p3 = np.array(
            [size, size, -(normal[0] * size + normal[1] * size + d) / normal[2]]
        )
        p4 = np.array(
            [-size, size, -(normal[0] * (-size) + normal[1] * size + d) / normal[2]]
        )

        # Criar os pontos para o mesh
        points = np.vstack((p1, p2, p3, p4))

        # Criar os triângulos que formam o plano
        triangles = [[0, 1, 2], [0, 2, 3]]  # Triângulo 1  # Triângulo 2

        # Criar a malha triangular
        plane_mesh = o3d.geometry.TriangleMesh()
        plane_mesh.vertices = o3d.utility.Vector3dVector(points)
        plane_mesh.triangles = o3d.utility.Vector3iVector(triangles)

        # Colorir a malha do plano de azul
        plane_mesh.paint_uniform_color([0.45, 0.45, 0.45])

        return plane_mesh

    def _create_axis_arrow(self, length=1.0, color=[1, 0, 0], rotation=None):
        arrow = o3d.geometry.TriangleMesh.create_arrow(
            cylinder_radius=0.01,
            cone_radius=0.03,
            cylinder_height=length * 0.8,
            cone_height=length * 0.2,
        )
        arrow.compute_vertex_normals()
        arrow.paint_uniform_color(color)
        if rotation is not None:
            arrow.rotate(rotation, center=(0, 0, 0))
        return arrow

    def show(self, *args, **kwargs):
        if self.cluster_viz:
            self.show_clusters(*args, **kwargs)
        else:
            self.show_point_cloud(*args, **kwargs)

    def show_clusters(
        self,
        clusters: np.ndarray,
        show_true_label: bool = False,
        show_grid: bool = False,
    ) -> None:
        pass

    def show_point_cloud(
        self,
        point_cloud: np.ndarray,
        normal_d_tuple: tuple | None = None,
        show_true_label: bool = False,
        show_ground: bool = True,
        show_clusters: bool = True,
        show_unlabeled: bool = True,
        show_plane: bool = False,
        show_grid: bool = False,
    ) -> None:
        """
        Visualize the filtered point cloud using Open3D.

        Args:
            point_cloud (np.ndarray): N x 6 array [x, y, z, true_label, pred_label, scanline_id].
        """

        label_col = 3 if show_true_label else 4
        labels = point_cloud[:, label_col]

        # Apply filter mask
        mask = (
            (show_plane & (labels == -1))
            | (show_unlabeled & (labels == 0))
            | (show_ground & (labels == 9))
            | (show_clusters & (labels >= 1) & (labels != 9))
        )

        xyz = point_cloud[mask, :3]
        visible_labels = labels[mask].astype(int)
        colors = self._get_color_map(visible_labels)

        # Visualize with point size
        vis = o3d.visualization.Visualizer()  # type: ignore
        vis.create_window(window_name="Plane Visualization", width=800, height=600)

        opt = vis.get_render_option()
        opt.point_size = self.point_size
        opt.background_color = np.asarray([0.1, 0.1, 0.1])  # estilo AutoCAD / pptk
        opt.show_coordinate_frame = True
        opt.mesh_show_back_face = True  # Exibir o lado de trás do plano

        # extrair para uma funcao
        # X (vermelho): rotaciona -90° ao redor Z
        arrow_x = self._create_axis_arrow(
            length=1.0,
            color=[1, 0, 0],
            rotation=o3d.geometry.get_rotation_matrix_from_xyz([0, np.pi / 2, 0]),
        )
        # Y (verde): rotaciona +90° ao redor X
        arrow_y = self._create_axis_arrow(
            length=1.0,
            color=[0, 1, 0],
            rotation=o3d.geometry.get_rotation_matrix_from_xyz([-np.pi / 2, 0, 0]),
        )
        # Z (azul): já está na direção Z por padrão
        arrow_z = self._create_axis_arrow(length=1.0, color=[0, 0, 1], rotation=None)
        vis.add_geometry(arrow_x)
        vis.add_geometry(arrow_y)
        vis.add_geometry(arrow_z)

        # Create Open3D point cloud
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(xyz)
        pcd.colors = o3d.utility.Vector3dVector(colors)
        vis.add_geometry(pcd)

        if show_grid:
            vis.add_geometry(
                self._create_grid(self.grid_size, self.grid_spacing)
            )  # grid grande com espaçamento 1m
            opt.line_width = self.grid_line_width
        if show_plane and normal_d_tuple != None:
            vis.add_geometry(self._create_plane(normal_d_tuple))

        vis.run()
        vis.destroy_window()

In [ ]:
# | export
def run_viz(point_cloud, normal_d_tuple=None, **visualizer_params):
    temp_dir = "temp_vis"
    os.makedirs(temp_dir, exist_ok=True)

    # Salvar nuvem de pontos
    np.save(os.path.join(temp_dir, "point_cloud.npy"), point_cloud)

    # Salvar plano (normal + d)
    if normal_d_tuple:
        normal_d = np.array([*normal_d_tuple[0], normal_d_tuple[1]])
        np.save(os.path.join(temp_dir, "normal_d.npy"), normal_d)

    # Salvar parâmetros
    with open(os.path.join(temp_dir, "visualizer_config.json"), "w") as f:
        json.dump(visualizer_params, f)

    # Rodar visualizador em subprocesso chamando o próprio utils.py
    comando = ["python3", "../pfc_packages/utils.py", temp_dir]
    thread = threading.Thread(target=lambda: subprocess.run(comando))
    thread.start()

In [ ]:
# | export
def main_viz(temp_dir):
    # Carrega dados salvos no diretório temporário
    point_cloud = np.load(os.path.join(temp_dir, "point_cloud.npy"))

    normal_d_path = os.path.join(temp_dir, "normal_d.npy")
    if os.path.exists(normal_d_path):
        normal_d_arr = np.load(normal_d_path)
        normal_d_tuple = (normal_d_arr[:3], normal_d_arr[3])
    else:
        normal_d_tuple = None

    with open(os.path.join(temp_dir, "visualizer_config.json")) as f:
        config = json.load(f)

    visualizer = PointCloudVisualizer(
        point_size=config.get("point_size", 1.0),
        grid_size=config.get("grid_size", 50),
        grid_spacing=config.get("grid_spacing", 1.0),
        grid_line_width=config.get("grid_line_width", 10),
        cluster_viz=config.get("cluster_viz", False),
    )

    visualizer.show(
        point_cloud,
        normal_d_tuple=normal_d_tuple,
        show_plane=config.get("show_plane", False),
        show_grid=config.get("show_grid", False),
        show_ground=config.get("show_ground", True),
        show_clusters=config.get("show_clusters", True),
        show_unlabeled=config.get("show_unlabeled", True),
        show_true_label=config.get("show_true_label", False),
    )

    # Limpar diretório temporário
    if os.path.exists(temp_dir):
        shutil.rmtree(temp_dir)


if __name__ == "__main__":
    if len(sys.argv) > 1:
        main_viz(sys.argv[1])
    else:
        print("No temporary directory provided. Exiting.")

# Save

In [ ]:
# | export
def save_clusters(
    point_cloud: np.ndarray, seq: str, frame_id: str, output_base_path: str
) -> None:
    """
    Salva os rótulos preditos em formato .label seguindo a estrutura do SemanticKITTI.

    Args:
        point_cloud (np.ndarray): Array Nx6 com colunas [x, y, z, true_label, predicted_label, scanline_index].
        seq (str): Número da sequência (ex: '00', '01', ...).
        frame_id (str): ID do frame (ex: '000123').
        output_base_path (str): Caminho base até `data_odometry_cluster_pred/dataset/sequences`.
    """
    # Extrair coluna de predição (índice 4)
    predicted_labels = point_cloud[:, 4].astype(np.uint32)

    # Construir caminho de saída
    output_dir = Path(output_base_path) / seq / "clusters"
    output_dir.mkdir(parents=True, exist_ok=True)

    # Nome do arquivo com sufixo _pred.label
    output_file = output_dir / f"{frame_id}_cluster.label"

    # Salvar como arquivo binário .label
    predicted_labels.tofile(output_file)

# Result analisys

In [ ]:
# | export
def verificar_consistencia_labels(pontos):
    """
    Verifica inconsistências entre rótulos verdadeiros e clusters atribuídos.
    Também calcula estatísticas (média e desvio padrão) dos clusters inconsistentes.

    Entrada:
        pontos (np.ndarray): Array N x 6 contendo [x, y, z, true_label, pred_label, scanline_id].

    Retorna:
        - inconsistentes_total (int): Número de clusters inconsistentes encontrados.
        - combinacoes_contadas (Counter): Contagem de combinações de labels inconsistentes.
        - estatisticas_erro (dict): Médias e desvios padrão das métricas de erro, mais erro do cluster 9.
    """

    # Extrai o vetor de labels preditos dos pontos (coluna 4)
    labels_pred = pontos[:, 4].astype(int)  # Exemplo: [1, 1, 2, 2, 3]

    # Extrai o vetor de labels verdadeiros dos pontos (coluna 3)
    labels_true = pontos[:, 3].astype(int)  # Exemplo: [2, 2, 6, 6, 9]

    # Combina predição e verdade em pares (pred_label, true_label) para cada ponto
    pares = np.stack((labels_pred, labels_true), axis=1)
    # Exemplo de pares:
    # [[1, 2],
    #  [1, 2],
    #  [2, 6],
    #  [2, 6],
    #  [3, 9]]

    # Cria um dicionário para mapear cada cluster predito para a lista de true_labels associados
    # Exemplo após o preenchimento:
    # cluster_to_true_labels = {
    #     1: [2, 2],
    #     2: [6, 6],
    #     3: [9]
    # }
    cluster_to_true_labels = defaultdict(list)
    for cluster_id, true_label in pares:
        cluster_to_true_labels[cluster_id].append(true_label)

    # Lista para armazenar combinações de labels inconsistentes
    combinacoes = []

    # Contador de clusters inconsistentes
    inconsistentes_total = 0

    # Listas para armazenar métricas de erro dos clusters inconsistentes
    lista_total_pontos = []  # Exemplo: [10, 15]
    lista_pontos_label_dominante = []  # Exemplo: [7, 12]
    lista_erro_percentual = []  # Exemplo: [30.0, 20.0]

    # Variável para armazenar o erro percentual do cluster 9 (solo)
    erro_percentual_cluster_9 = None

    # Itera sobre cada cluster_id e sua lista de true_labels
    for cluster_id, labels in cluster_to_true_labels.items():

        # Ignora o cluster 9, pois será tratado separadamente
        if cluster_id == 9:
            continue

        # Remove pontos sem rótulo (label 0)
        labels_sem_unlabeled = [l for l in labels if l != 0]
        # Exemplo: se labels = [0, 2, 2, 3], labels_sem_unlabeled = [2, 2, 3]

        # Verifica se existem múltiplos labels (inconsistência no cluster)
        #
        # labels_sem_unlabeled é uma lista que contém os rótulos verdadeiros (true_labels) dos pontos de um cluster, ignorando os rótulos "0" (não rotulados).
        #
        # A função set() em Python transforma a lista em um conjunto, ou seja, remove todos os elementos repetidos.
        # Por exemplo:
        #   labels_sem_unlabeled = [2, 2, 2] → set(labels_sem_unlabeled) = {2} (apenas um elemento)
        #   labels_sem_unlabeled = [2, 2, 3] → set(labels_sem_unlabeled) = {2, 3} (dois elementos diferentes)
        #
        # Depois, usamos len(set(...)) para contar quantos elementos únicos existem.
        #
        # Se existir mais de 1 label diferente (len > 1), isso significa que o cluster está inconsistente — porque ele deveria ter apenas pontos de um único tipo (um único rótulo verdadeiro).
        if len(set(labels_sem_unlabeled)) > 1:
            inconsistentes_total += 1  # Incrementa o contador

            # Cria uma combinação ordenada dos labels para contagem
            combinacao = tuple(sorted(set(labels_sem_unlabeled)))
            # Exemplo: (2, 3)

            combinacoes.append(combinacao)

            # Estatísticas para o cluster inconsistente:

            # Número total de pontos no cluster
            total_pontos = len(labels)

            # Conta quantos pontos existem para cada true_label
            #
            # labels_sem_unlabeled é a lista de rótulos verdadeiros dos pontos dentro de um cluster (ignorando os 0s).
            #
            # Usamos Counter(labels_sem_unlabeled) para contar quantas vezes cada label aparece.
            # Counter é uma ferramenta do Python que cria automaticamente um "dicionário de contagem".
            #
            # Exemplo:
            # labels_sem_unlabeled = [2, 2, 3]
            # Então, contador_labels = Counter({2: 2, 3: 1})
            # Significa que o label 2 aparece 2 vezes e o label 3 aparece 1 vez.
            contador_labels = Counter(labels_sem_unlabeled)

            # Identifica o label dominante (aquele com mais pontos)
            #
            # Depois de contar quantos pontos existem para cada label, queremos saber qual label aparece mais vezes — o dominante.
            #
            # Usamos contador_labels.most_common(1) para pegar o label mais frequente.
            # Isso retorna uma lista com o (label, quantidade), e o [0] serve para pegar o primeiro (e único) elemento dessa lista.
            #
            # Exemplo:
            # contador_labels = Counter({2: 2, 3: 1})
            # contador_labels.most_common(1) = [(2, 2)]
            # Ou seja:
            #   label_dominante = 2  (o label que mais aparece)
            #   pontos_label_dominante = 2  (quantas vezes ele aparece)
            label_dominante, pontos_label_dominante = contador_labels.most_common(1)[0]

            # Calcula o erro percentual (percentual de pontos errados)
            erro_percentual = 100 * (1 - (pontos_label_dominante / total_pontos))
            # Exemplo: erro_percentual = 100 * (1 - 2/3) ≈ 33.33%

            # Armazena para cálculo posterior de média e desvio padrão
            lista_total_pontos.append(total_pontos)
            lista_pontos_label_dominante.append(pontos_label_dominante)
            lista_erro_percentual.append(erro_percentual)

    # Conta quantas vezes cada combinação de labels inconsistentes apareceu
    combinacoes_contadas = Counter(combinacoes)
    # Exemplo: Counter({(2, 3): 1, (6, 7): 2})

    # ---- Cálculo especial para o cluster 9 (chão) ----

    # Pega todos os true_labels associados ao cluster 9
    labels = cluster_to_true_labels[9]
    # Exemplo: labels = [9, 9, 9, 5] (erro no último)

    # Número total de pontos no cluster 9
    total_pontos_cluster_9 = len(labels)

    # Conta quantos pontos têm erro (não são 9)
    num_erros_cluster_9 = sum(1 for l in labels if l != 9)

    # Calcula o erro percentual do cluster 9
    if total_pontos_cluster_9 > 0:
        erro_percentual_cluster_9 = 100 * (num_erros_cluster_9 / total_pontos_cluster_9)
        # Exemplo: 1 erro em 4 pontos → 25%
    else:
        erro_percentual_cluster_9 = 0.0

    # ---- Cálculo das estatísticas gerais ----

    # Se existirem clusters inconsistentes
    if inconsistentes_total > 0:
        estatisticas_erro = {
            "media_total_pontos": np.mean(
                lista_total_pontos
            ),  # Média do total de pontos nos clusters inconsistentes
            "std_total_pontos": np.std(
                lista_total_pontos
            ),  # Desvio padrão do total de pontos
            "media_pontos_label_dominante": np.mean(
                lista_pontos_label_dominante
            ),  # Média de pontos no label dominante
            "std_pontos_label_dominante": np.std(
                lista_pontos_label_dominante
            ),  # Desvio padrão dos pontos dominantes
            "media_erro_percentual": np.mean(
                lista_erro_percentual
            ),  # Média dos erros percentuais
            "std_erro_percentual": np.std(
                lista_erro_percentual
            ),  # Desvio padrão dos erros percentuais
            "erro_percentual_cluster_9": erro_percentual_cluster_9,  # Erro no cluster do chão
        }
    else:
        # Se não houver inconsistências, inicializa as métricas com zero
        estatisticas_erro = {
            "media_total_pontos": 0,
            "std_total_pontos": 0,
            "media_pontos_label_dominante": 0,
            "std_pontos_label_dominante": 0,
            "media_erro_percentual": 0,
            "std_erro_percentual": 0,
            "erro_percentual_cluster_9": erro_percentual_cluster_9,
        }

    # Retorna o número de inconsistências, as combinações contadas e as estatísticas calculadas
    return inconsistentes_total, combinacoes_contadas, estatisticas_erro

In [ ]:
# | export
def get_statistics(idx, point_cloud):
    clusters = point_cloud[:, 4]
    num_clusters = len(np.unique(clusters))
    num_pontos = point_cloud.shape[0]

    cluster_sizes = [np.sum(clusters == label) for label in np.unique(clusters)]
    cluster_sizes_sorted = sorted(cluster_sizes, reverse=True)

    ground_cluster_size = np.sum(clusters == 9)
    largest_non_ground_cluster = (
        cluster_sizes_sorted[1]
        if cluster_sizes_sorted[0] == ground_cluster_size
        and len(cluster_sizes_sorted) > 1
        else cluster_sizes_sorted[0]
    )

    num_clusters_inconsistentes, combinacoes, erros_clusters = (
        verificar_consistencia_labels(point_cloud)
    )

    estatisticas_do_frame = {
        "frame_id": idx,
        "num_clusters": num_clusters,
        "num_pontos": num_pontos,
        "pontos_por_cluster_medio": (
            num_pontos / num_clusters if num_clusters > 0 else 0
        ),
        "largest_non_ground_cluster": largest_non_ground_cluster,
        "gound_cluster_size": ground_cluster_size,
        "num_clusters_inconsistentes": num_clusters_inconsistentes,  # Novo campo
        "combinacoes": combinacoes,
        "erros_clusters": erros_clusters,
    }

    return estatisticas_do_frame

# Plots

In [ ]:
# | export
def _plot_with_stats(
    ax,
    x,
    y,
    color,
    label,
    title,
    ylabel,
    linestyle="o",
    mean_fmt="{:.1f}",
    std_fmt="{:.1f}",
):
    """Função auxiliar para plotar métricas com média e desvio padrão."""
    media = np.mean(y)
    desvio = np.std(y)
    ax.plot(x, y, marker=linestyle, label=label, color=color)
    ax.axhline(
        media, color=color, linestyle="--", label=f"Média = {mean_fmt.format(media)}"
    )
    ax.fill_between(
        x,
        media - desvio,
        media + desvio,
        color=color,
        alpha=0.2,
        label=f"Desvio = {std_fmt.format(desvio)}",
    )
    ax.set_title(title)
    ax.set_xlabel("Frame")
    ax.set_ylabel(ylabel)
    ax.legend()
    ax.grid(axis="y", color="gray", linestyle="--", linewidth=0.7, alpha=0.4)

In [ ]:
# | export
def plot_1(resumo_por_frame, seq_value=None):
    """
    Plota em 4 subplots verticais:
    1. Número total de pontos
    2. Número total de clusters
    3. Pontos por cluster médio
    4. Porcentagem de redução da dimensionalidade
    """
    num_clusters = np.array([frame["num_clusters"] for frame in resumo_por_frame])
    num_pontos = np.array([frame["num_pontos"] for frame in resumo_por_frame])
    pontos_por_cluster_medio = np.array(
        [frame["pontos_por_cluster_medio"] for frame in resumo_por_frame]
    )
    frames = np.arange(len(resumo_por_frame))
    reducao_dimensionalidade = 1 - (num_clusters / num_pontos)

    fig, axs = plt.subplots(4, 1, figsize=(12, 18))
    fig.tight_layout(pad=5.0)

    _plot_with_stats(
        axs[0],
        frames,
        num_pontos,
        "green",
        "Número de Pontos",
        "Número de Pontos por Frame",
        "Número de Pontos",
    )
    _plot_with_stats(
        axs[1],
        frames,
        num_clusters,
        "blue",
        "Número de Clusters",
        "Número de Clusters por Frame",
        "Número de Clusters",
    )
    _plot_with_stats(
        axs[2],
        frames,
        pontos_por_cluster_medio,
        "orange",
        "Pontos por Cluster Médio",
        "Média de Pontos por Cluster por Frame",
        "Pontos por Cluster",
    )
    _plot_with_stats(
        axs[3],
        frames,
        reducao_dimensionalidade * 100,
        "purple",
        "Redução Dimensionalidade (%)",
        "Porcentagem de Redução da Dimensionalidade por Frame",
        "Redução (%)",
        mean_fmt="{:.2f}",
        std_fmt="{:.2f}",
    )

    plt.tight_layout()
    plt.show()

    if seq_value is not None:
        plot_dir = Path("plots") / str(seq_value)
        plot_dir.mkdir(parents=True, exist_ok=True)
        fig.savefig(plot_dir / "plot-1.png", dpi=300, bbox_inches="tight")

In [ ]:
# | export
def plot_2(resumo_por_frame, seq_value=None):
    ground_cluster_size = [f["gound_cluster_size"] for f in resumo_por_frame]
    largest_non_ground_cluster = [
        f["largest_non_ground_cluster"] for f in resumo_por_frame
    ]
    frames = np.arange(len(resumo_por_frame))

    fig, axs = plt.subplots(2, 1, figsize=(12, 10), sharex=True)
    _plot_with_stats(
        axs[0],
        frames,
        ground_cluster_size,
        "green",
        "Ground cluster",
        "Tamanho do Ground Cluster",
        "Tamanho",
    )
    _plot_with_stats(
        axs[1],
        frames,
        largest_non_ground_cluster,
        "blue",
        "Maior non ground cluster",
        "Tamanho do Maior non Ground Cluster",
        "Tamanho",
    )

    plt.tight_layout()
    plt.show()
    if seq_value is not None:
        plot_dir = Path("plots") / str(seq_value)
        plot_dir.mkdir(parents=True, exist_ok=True)
        fig.savefig(plot_dir / "plot-2.png", dpi=300, bbox_inches="tight")

In [ ]:
# | export
def plot_3(resumo_por_frame, seq_value=None):
    inconsistentes_por_frame = np.array(
        [frame["num_clusters_inconsistentes"] for frame in resumo_por_frame]
    )
    num_clusters_por_frame = np.array(
        [frame["num_clusters"] for frame in resumo_por_frame]
    )
    frames = np.arange(len(resumo_por_frame))

    porcentagem_erro = 1 - (inconsistentes_por_frame / num_clusters_por_frame)
    porcentagem_erro = np.clip(porcentagem_erro, 0, 1)

    fig, axs = plt.subplots(2, 1, figsize=(10, 10), sharex=True)
    _plot_with_stats(
        axs[0],
        frames,
        inconsistentes_por_frame,
        "red",
        "Inconsistências por frame",
        "Clusters Inconsistentes por Frame",
        "Nº de Clusters Inconsistentes",
        mean_fmt="{:.2f}",
        std_fmt="{:.2f}",
    )
    _plot_with_stats(
        axs[1],
        frames,
        porcentagem_erro,
        "green",
        "Porcentagem de acerto",
        "Porcentagem de Erro por Frame",
        "Porcentagem de Erro",
        mean_fmt="{:.2f}",
        std_fmt="{:.2f}",
    )

    plt.tight_layout()
    plt.show()
    if seq_value is not None:
        plot_dir = Path("plots") / str(seq_value)
        plot_dir.mkdir(parents=True, exist_ok=True)
        fig.savefig(plot_dir / "plot-3.png", dpi=300, bbox_inches="tight")

In [ ]:
# | export
def plot_4(resumo_por_frame, seq_value=None):
    """
    Plota sete métricas de erro em subplots separados no mesmo figure.
    """
    # Cada métrica: (chave_media, chave_std, título_média, título_std, ylabel)
    metricas = [
        (
            "media_total_pontos",
            "std_total_pontos",
            "Média: Total de Pontos nos Clusters Inconsistentes por Frame",
            "Desvio: Total de Pontos nos Clusters Inconsistentes por Frame",
            "Total de Pontos",
        ),
        (
            "media_pontos_label_dominante",
            "std_pontos_label_dominante",
            "Média: Pontos do Label Dominante por Cluster Inconsistente",
            "Desvio: Pontos do Label Dominante por Cluster Inconsistente",
            "Pontos do Label Dominante",
        ),
        (
            "media_erro_percentual",
            "std_erro_percentual",
            "Média: Erro Percentual nos Clusters Inconsistentes por Frame",
            "Desvio: Erro Percentual nos Clusters Inconsistentes por Frame",
            "Erro Percentual (%)",
        ),
        (
            "erro_percentual_cluster_9",
            None,
            "Erro Percentual no Cluster de Solo (Ground) por Frame",
            None,
            "Erro Percentual no Solo (%)",
        ),
    ]
    frames = np.arange(len(resumo_por_frame))
    fig, axs = plt.subplots(7, 1, figsize=(14, 28))
    fig.tight_layout(pad=5.0)
    plot_idx = 0

    for chave_media, chave_std, titulo_media, titulo_std, ylabel in metricas:
        # Plot média
        valores_media = np.array(
            [frame["erros_clusters"][chave_media] for frame in resumo_por_frame]
        )
        _plot_with_stats(
            axs[plot_idx],
            frames,
            valores_media,
            "red",
            f"{ylabel} (média) por frame",
            titulo_media,
            ylabel,
            mean_fmt="{:.2f}",
            std_fmt="{:.2f}",
        )
        plot_idx += 1
        # Plot std se existir
        if chave_std:
            valores_std = np.array(
                [frame["erros_clusters"][chave_std] for frame in resumo_por_frame]
            )
            _plot_with_stats(
                axs[plot_idx],
                frames,
                valores_std,
                "purple",
                f"{ylabel} (desvio) por frame",
                titulo_std,
                f"Desvio de {ylabel}",
                mean_fmt="{:.2f}",
                std_fmt="{:.2f}",
            )
            plot_idx += 1

    plt.tight_layout()
    plt.show()
    if seq_value is not None:
        plot_dir = Path("plots") / str(seq_value)
        plot_dir.mkdir(parents=True, exist_ok=True)
        fig.savefig(plot_dir / "plot-4.png", dpi=300, bbox_inches="tight")

In [ ]:
# | export
def plot_5(combinacoes_geral, seq_value=None, top_n=15):
    label_id_to_name = {
        -1: "plane",
        0: "unlabeled",
        1: "car",
        2: "bicycle",
        3: "motorcycle",
        4: "truck",
        5: "other-vehicle",
        6: "person",
        7: "bicyclist",
        8: "motorcyclist",
        9: "road",
        10: "parking",
        11: "sidewalk",
        12: "other-ground",
        13: "building",
        14: "fence",
        15: "vegetation",
        16: "trunk",
        17: "terrain",
        18: "pole",
        19: "traffic-sign",
    }

    combinacoes_ordenadas = sorted(
        combinacoes_geral.items(), key=lambda x: x[1], reverse=True
    )
    top_combinacoes = combinacoes_ordenadas[:top_n]


    def formatar_combinacao(combinacao):
        nomes = [label_id_to_name.get(int(l), f"desconhecido({l})") for l in combinacao]
        return ", ".join(nomes)

    labels_pizza = [formatar_combinacao(comb) for comb, _ in top_combinacoes]
    valores_pizza = [contagem for _, contagem in top_combinacoes]

    total = sum(combinacoes_geral.values())
    total_top = sum(valores_pizza)
    outros = total - total_top

    if outros > 0:
        labels_pizza.append("Outros")
        valores_pizza.append(outros)

    colors = cm.get_cmap("tab20")(np.linspace(0, 1, len(labels_pizza)))

    fig, ax = plt.subplots(figsize=(12, 10))

    # Calcula as porcentagens de cada fatia
    valores_pizza = np.array(valores_pizza)
    porcentagens = 100 * valores_pizza / valores_pizza.sum()

    # Função para mostrar porcentagem só se o slice for maior que 3%
    def autopct_format(pct):
        return ("%1.1f%%" % pct) if pct > 3 else ""

    wedges, texts, autotexts = ax.pie( # type: ignore
        valores_pizza,
        labels=None,  # Não mostra os labels diretamente
        autopct=autopct_format,
        startangle=140,
        colors=colors, # type: ignore
        pctdistance=0.8,
        wedgeprops=dict(edgecolor="w", linewidth=1),
        textprops=dict(fontsize=14, color="black"),
    )

    # Monta a legenda em colunas, porcentagens pequenas vão ao lado do nome
    legend_labels = []
    for nome, pct in zip(labels_pizza, porcentagens):
        legend_labels.append(f"{nome} ({pct:.1f}%)")

    # Calcula número de colunas para legenda (máx 5 por coluna)
    n_labels = len(legend_labels)
    ncol = min(3, max(1, (n_labels + 4) // 5))  # até 5 linhas por coluna

    ax.set_title(f"Top {top_n} combinações inconsistentes + Outros", fontsize=20, pad=20)
    ax.axis("equal")

    # Legenda embaixo, em colunas, centralizada
    fig.legend(
        wedges,
        legend_labels,
        title="Combinações",
        loc="lower center",
        bbox_to_anchor=(0.5, -0.08),
        fontsize=14,
        title_fontsize=16,
        frameon=False,
        ncol=ncol,
        columnspacing=1.5,
        handlelength=1.5,
    )

    plt.tight_layout(rect=[0, 0.08, 1, 1])  # type: ignore 

    plt.show()
    if seq_value is not None:
        Path("plots").mkdir(parents=True, exist_ok=True)
        Path(f"plots/{seq_value}").mkdir(parents=True, exist_ok=True)
        fig.savefig(f"plots/{seq_value}/plot-5.png", dpi=300, bbox_inches="tight")